In [ ]:
import requests_with_caching
import json

def get_movies_from_tastedive(movie_name):
    baseurl = 'https://tastedive.com/api/similar'
    parameter = {}
    parameter['q'] = movie_name
    parameter['type'] = 'movies'
    parameter['limit'] = 5
    movie_resp = requests_with_caching.get(baseurl, params = parameter)
    return movie_resp.json()

def extract_movie_titles(movie_dict):
    movie_name_lst = []
    for dict in movie_dict['Similar']['Results']:
        movie_name_lst.append(dict['Name'])
    return movie_name_lst

def get_related_titles(movie_title_lst):
    related_titles_lst = []
    for movie in movie_title_lst:
        related_titles = extract_movie_titles(get_movies_from_tastedive(movie))
        for title in related_titles:
            if title not in related_titles_lst:
                related_titles_lst.append(title)
    return related_titles_lst

def get_movie_data(title):
    baseurl = 'http://www.omdbapi.com/'
    parameter = {}
    parameter['t'] = title
    parameter['r'] = 'json'
    title_resp = requests_with_caching.get(baseurl, params = parameter)
    return title_resp.json()

def get_movie_rating(dict):
    for rating in dict['Ratings']:
        if rating['Source'] == 'Rotten Tomatoes':
            return int(rating['Value'][:-1])
    return 0

def get_sorted_recommendations(movie_titles):
    list = []
    for title in movie_titles:
        list = list + extract_movie_titles(get_movies_from_tastedive(title))
    score_list = sorted(list, reverse = True, key = lambda movie: (get_movie_rating(get_movie_data(movie)), movie))
    return score_list